In [ ]:
!wget https://huggingface.co/UBC-NLP/MARBERT/resolve/main/MARBERT_pytorch_verison.tar.gz

--2021-06-09 15:24:39--  https://huggingface.co/UBC-NLP/MARBERT/resolve/main/MARBERT_pytorch_verison.tar.gz
Resolving huggingface.co (huggingface.co)... 15.197.130.34
Connecting to huggingface.co (huggingface.co)|15.197.130.34|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/UBC-NLP/MARBERT/85bfec76f38cba4bc2e6cd02a959016de37ba93de4c850a7d175811dce4e8adc [following]
--2021-06-09 15:24:40--  https://cdn-lfs.huggingface.co/UBC-NLP/MARBERT/85bfec76f38cba4bc2e6cd02a959016de37ba93de4c850a7d175811dce4e8adc
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 99.86.113.22, 99.86.113.119, 99.86.113.117, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|99.86.113.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607066087 (579M) [application/x-gzip]
Saving to: ‘MARBERT_pytorch_verison.tar.gz’

MARBERT_pytorch_ver 100%[===================>] 578.94M   143MB/s    in 4.1s    

2021-06-09 

In [ ]:
!tar -xvf MARBERT_pytorch_verison.tar.gz

MARBERT_pytorch_verison/
MARBERT_pytorch_verison/pytorch_model.bin
MARBERT_pytorch_verison/config.json
MARBERT_pytorch_verison/vocab.txt


In [ ]:
!pip install GPUtil pytorch_pretrained_bert==0.5.0 transformers==4.3.0

     |████████████████████████████████| 102kB 6.7MB/s 
     |████████████████████████████████| 1.8MB 23.0MB/s 
     |████████████████████████████████| 133kB 36.6MB/s 
     |████████████████████████████████| 901kB 35.0MB/s 
     |████████████████████████████████| 3.3MB 35.4MB/s 
     |████████████████████████████████| 81kB 10.2MB/s 
     |████████████████████████████████| 7.6MB 33.3MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=d048d797f935bc6f060abde9f787f358ee0c1a5e3912158a48707982e06369b4
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil
ERROR: botocore 1.20.90 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
# (1)load libraries 
import json, sys, regex
import torch
import GPUtil
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
##----------------------------------------------------
from transformers import *
from transformers import XLMRobertaConfig
from transformers import XLMRobertaModel
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, XLMRobertaModel
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel
##------------------------------------------------------
import re

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [ ]:
MODEL_PATH_BEGIN_FINETUNE='/content/MARBERT_pytorch_verison'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("your device ", device)

your device  cuda


In [125]:
class SentimentIdentificationMARBERT(object):
  """A class for finetunning, evaluating and running the sentiment classification on MARBERT model
     After initializing an instance, you must
    run the train method once before using it.
    Args:
        labels (:obj:`set` of :obj:`str`, optional): The set of sentiment labels
            used in the training data in the main model.
            If None, the default labels are used.
            Defaults to None.

        max_seq_length (:obj:`int`, optional): maximum sequence length for the model
            
            If None, the default max_seq_length are used.
            Defaults to 256 .

        num_epoch (:obj:`int`, optional): number of epoch used for training the model
            
            If None, the default num_epoch are used.
            Defaults to 3 .

        batch_size (:obj:`int`, optional):batch size used for training the model            
            If None, the default batch_size are used.
            Defaults to 16 .

        lr (:obj:`int`, optional):initial learning rate used for training the model            
            If None, the default lr are used.
            Defaults to 2e-06 .
        
       
    """
  def __init__(self, labels=None,max_seq_length=256,num_epoch=3,batch_size=16,lr=2e-06
                 ):
        if labels is None:
            self.labels = _DEFAULT_LABELS
        self._labels_sorted = sorted(labels)
        self._is_trained = False
        self.tokenizer = BertTokenizer.from_pretrained(MODEL_PATH_BEGIN_FINETUNE)
        self.num_epoch=num_epoch
        self.batch_size=batch_size
        self.max_seq_length=max_seq_length
        self.lr=lr
  def create_label2ind_file(self):
      self.labels_json = {}
      # convert labels to indexes
      for idx in range(0, len(self._labels_sorted)):
        self.labels_json[self._labels_sorted[idx]] = idx
  def save_label2ind_file(self,path):
    """Save  the label 2 indexr on a given data set.
      Args:
          Path (:obj:`str`): Path where you want to save the feature vector .
              
          
      """
    with open(path, 'w') as json_file:
        json.dump(self.labels_json, json_file)

  def data_prepare_BERT(self,X_train,y_train):
      # Create sentence and label lists
      sentences = X_train.values
      sentences = ['[CLS] ' + sentence + ' [SEP]' for sentence in
                 sentences]
      # Create sentence and label lists
      labels = y_train.values
      labels = [self.labels_json[i] for i in labels]
      # Import the BERT tokenizer, used to convert our text into tokens that correspond to BERT's vocabulary.
      tokenized_texts = [self.tokenizer.tokenize(sent) for sent in sentences]
      # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
      input_ids = [self.tokenizer.convert_tokens_to_ids(x) for x in
                 tokenized_texts]
      # Pad our input seqeunce to the fixed length (i.e., max_len) with index of [PAD] token
      # ~ input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
      pad_ind = self.tokenizer.convert_tokens_to_ids(['[PAD]'])[0]
      input_ids = pad_sequences(
          input_ids,
          maxlen=self.max_seq_length + 2,
          dtype='long',
          truncating='post',
          padding='post',
          value=pad_ind,
          )
      # Create attention masks
      attention_masks = []
      # Create a mask of 1s for each token followed by 0s for padding
      for seq in input_ids:
        seq_mask = [float(i > 0) for i in seq]
        attention_masks.append(seq_mask)
      # Convert all of our data into torch tensors, the required datatype for our model
      inputs = torch.tensor(input_ids)
      labels = torch.tensor(labels)
      masks = torch.tensor(attention_masks)
      return inputs, labels, masks
  def data_prepare_BERT_test(self,X_train):
      # Create sentence and label lists
      sentences = X_train.values
      sentences = ['[CLS] ' + sentence + ' [SEP]' for sentence in
                 sentences]
     
      # Import the BERT tokenizer, used to convert our text into tokens that correspond to BERT's vocabulary.
      tokenized_texts = [self.tokenizer.tokenize(sent) for sent in sentences]
      # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
      input_ids = [self.tokenizer.convert_tokens_to_ids(x) for x in
                 tokenized_texts]
      # Pad our input seqeunce to the fixed length (i.e., max_len) with index of [PAD] token
      # ~ input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
      pad_ind = self.tokenizer.convert_tokens_to_ids(['[PAD]'])[0]
      input_ids = pad_sequences(
          input_ids,
          maxlen=self.max_seq_length + 2,
          dtype='long',
          truncating='post',
          padding='post',
          value=pad_ind,
          )
      # Create attention masks
      attention_masks = []
      # Create a mask of 1s for each token followed by 0s for padding
      for seq in input_ids:
        seq_mask = [float(i > 0) for i in seq]
        attention_masks.append(seq_mask)
      # Convert all of our data into torch tensors, the required datatype for our model
      inputs = torch.tensor(input_ids)
      masks = torch.tensor(attention_masks)
      return inputs,  masks
  def train(self,train_dataloader, optimizer, scheduler, criterion,max_grad_norm):
    self.model.train()
    epoch_loss = 0
    for i, batch in enumerate(train_dataloader):
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      input_ids, input_mask, labels = batch
      outputs = self.model(input_ids, input_mask, labels=labels)
      loss, logits = outputs[:2]
      # delete used variables to free GPU memory
      del batch, input_ids, input_mask, labels
      optimizer.zero_grad()
      if torch.cuda.device_count() == 1:
        loss.backward()
        epoch_loss += loss.cpu().item()
      else:

        loss.sum().backward()
         
        epoch_loss += loss.sum().cpu().item()
      optimizer.step()
      torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore
      scheduler.step()
    # free GPU memory
    if device == 'cuda':
      torch.cuda.empty_cache()
    return epoch_loss / len(train_dataloader)
  def save_model(self,path):   
    """Save  the model on a given data set.
        Args:
            Path (:obj:`str`): Path where you want to save the model.
               
           
        """
    self.model.save_pretrained(path + '/')

  def eval(self,X_eval,y_eval, data_set='DEV'):

    """Evaluate the trained model on a given data set.
        Args:
            X_eval (:obj:`np array or pandas series`, optional): loaded data for evaluation.

            y_eval (:obj:`np array or pandas series`, optional): loaded labels for evaluation.

            data_set (:obj:`str`, optional): Name of the provided data set to
                use. This is ignored if data_path is not None. Can be either
                'VALIDATION' or 'TEST'. Defaults to 'VALIDATION'.

            batch_size (:obj:`int`, optional):batch size used for training the model            
            If None, the default batch_size are used.
            Defaults to 16 .
        Returns:
            :obj:`dict`: A dictionary mapping an evaluation metric to its
            computed value. The metrics used are accuracy, f1_micro, f1_macro,
            recall_micro, recall_macro, precision_micro and precision_macro.
        """
    validation_inputs, validation_labels, validation_masks = self.data_prepare_BERT(X_eval,y_eval)
    validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
    validation_dataloader = DataLoader(validation_data, batch_size=self.batch_size)
    self.model.eval()
    all_pred=[]
    all_label = []
    with torch.no_grad():
      for i, batch in enumerate(validation_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        input_ids, input_mask, labels = batch
        outputs = self.model(input_ids, input_mask, labels=labels)
        loss, logits = outputs[:2]
        
        probabilities, predicted = torch.max(logits.cpu().data, 1)
        # put all the true labels and predictions to two lists
        all_pred.extend(predicted)
        all_label.extend(labels.cpu())
    accuracy = accuracy_score(all_label, all_pred)
    macro_f1_pos_neg = f1_score(all_label, all_pred,average='macro',labels=[0,1])
    f1score = f1_score(all_label, all_pred, average='macro') 
    recall = recall_score(all_label, all_pred, average='macro')
    precision = precision_score(all_label, all_pred, average='macro')
    # Get scores
    scores = {
        'Sentiment': {
            'accuracy': accuracy,
            'f1_macro': f1score,
            'recall_macro': recall,
            'precision_macro':precision
        }
    }
    return scores

  def predict(self,sentences):
    """Predict the sentiment  probability scores for a given list of
        sentences.
        Args:
            sentences (:obj:`list` of :obj:`str`): The list of sentences.
            output (:obj:`str`): The output label type. Possible values are
                'postive', 'neagtive', 'neutral'.
        Returns:
            :obj:`list` of :obj:`DIDPred`: A list of prediction results,
            each corresponding to its respective sentence.
        """
    if isinstance(sentences, str):
      sentences=pd.Series(sentences)
    validation_inputs, validation_masks = self.data_prepare_BERT_test(sentences)
    validation_data = TensorDataset(validation_inputs, validation_masks)
    validation_dataloader = DataLoader(validation_data, batch_size=self.batch_size)
    self.model.eval()
    all_pred=[]
    result = collections.deque()
    convert = lambda x: x
    
    with torch.no_grad():
      for i, batch in enumerate(validation_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        input_ids, input_mask = batch
        outputs = self.model(input_ids.to(device), input_mask.to(device))
        logits = outputs[:2][0]
        
        
        
        probabilities, predicted = torch.max(outputs[0].cpu().data, 1)
        for i, val in enumerate(self.label2index):
          if i==predicted.item():
            all_pred.extend((val,probabilities.item()))
            result.append(convert(DIDPred(val, logits.cpu().data.numpy().tolist())))
            break
        # put all the true labels and predictions to two lists

        
      
    return list(result)
      

        
  def evaluate(self,validation_dataloader, criterion):
    
    self.model.eval()
    epoch_loss = 0
    all_pred=[]
    all_label = []
    with torch.no_grad():
      for i, batch in enumerate(validation_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        input_ids, input_mask, labels = batch
        outputs = self.model(input_ids, input_mask, labels=labels)
        loss, logits = outputs[:2]
        # delete used variables to free GPU memory
        del batch, input_ids, input_mask
        if torch.cuda.device_count() == 1:
          epoch_loss += loss.cpu().item()
        else:
          epoch_loss += loss.sum().cpu().item()
        # identify the predicted class for each example in the batch
        probabilities, predicted = torch.max(logits.cpu().data, 1)
        # put all the true labels and predictions to two lists
        all_pred.extend(predicted)
        all_label.extend(labels.cpu())
    accuracy = accuracy_score(all_label, all_pred)
    f1score = f1_score(all_label, all_pred, average='macro')
    recall = recall_score(all_label, all_pred, average='macro')
    precision = precision_score(all_label, all_pred, average='macro')
    report = classification_report(all_label, all_pred)
    print('Accuracy', accuracy)
    print('f1score', f1score)
    print('recall', recall)
    print('precision', precision)
    print('report', report)
    return (epoch_loss / len(validation_dataloader)), accuracy, f1score, recall, precision

         

  def fine_tune(self,X_train,y_train,X_valid=None,y_valid=None):

    """ fine tune MARBERT model.
        Args:
            X_train (:obj:`np array or pandas series`, optional): loaded training data.
               
            y_train (:obj:`np array or pandas series`, optional): loaded labels for training.

            X_valid (:obj:`np array or pandas series`, optional): loaded validation data.
               
            y_valid (:obj:`np array or pandas series`, optional): loaded labels for validation.
       
        """

    self.create_label2ind_file()
    if X_valid is None and y_valid is None:
      msk = np.random.rand(len(X_train)) < 0.8
      X_valid = X_train[~msk]
      X_train = X_train[msk]
      y_valid = y_train[~msk]
      y_train = y_train[msk]
    #-------------------------------------------------------
    train_inputs, train_labels, train_masks = self.data_prepare_BERT(X_train,y_train)
    validation_inputs, validation_labels, validation_masks = self.data_prepare_BERT(X_valid,y_valid)
    #-------------------------------------------------------
    # Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
    self.model = BertForSequenceClassification.from_pretrained(MODEL_PATH_BEGIN_FINETUNE, num_labels=len(self.labels_json))
    #-------------------------------------------------------
    train_data = TensorDataset(train_inputs, train_masks, train_labels)
    train_dataloader = DataLoader(train_data, batch_size=self.batch_size)
    #-------------------------------------------------------
    validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
    validation_dataloader = DataLoader(validation_data, batch_size=self.batch_size)
    #------------------------------------------
    if torch.cuda.is_available():
      if torch.cuda.device_count() == 1:
        print("Run", "with one GPU")
        self.model = self.model.to(device)
      else:
        n_gpu = torch.cuda.device_count()
        print("Run", "with", n_gpu, "GPUs with max 4 GPUs")
        device_ids = GPUtil.getAvailable(limit = 4)
        torch.backends.cudnn.benchmark = True
        self.model = self.model.to(device)
        self.model = nn.DataParallel(model, device_ids=device_ids)
    else:

      print("Run", "with CPU")
      self.model = self.model
    #---------------------------------------------------
    max_grad_norm = 1.0
    warmup_proportion = 0.1
    num_training_steps	= len(train_dataloader) * self.num_epoch
    num_warmup_steps = num_training_steps * warmup_proportion
    optimizer = AdamW(self.model.parameters(), lr=self.lr, correct_bias=False)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)
    criterion = nn.CrossEntropyLoss()
    for epoch in trange(self.num_epoch, desc="Epoch"):
      train_loss = self.train( train_dataloader, optimizer, scheduler, criterion,max_grad_norm)
      val_loss, val_acc, val_f1, val_recall, val_precision = self.evaluate(validation_dataloader, criterion)
  

      





In [51]:
MODEL_PATH_='/content/drive/MyDrive/Omdena_sentiment/Saved_models/Production/MARBERT'
LABEL_2_INDEX_PATH='/content/drive/MyDrive/Omdena_sentiment/Saved_models/Production/MARBERT/_labels-dict.json'

In [127]:
import collections
class DIDPred(collections.namedtuple('SentimentPred', ['top', 'scores'])):
    """A named tuple containing sentiment ID prediction results.
    Attributes:
        top (:obj:`str`): The sentiment label with the highest score. See
            :ref:`sentimentid_labels` for a list of output labels.
        scores (:obj:`dict`): A dictionary mapping each sentiment label to it's
            computed score.
    """

In [121]:
class SentimentIdentificationMARBERTPrediction(object):
  """A class for running a fine-tuned sentiment analysis model to predict
    the sentiment of given sentences.


    Args:
        labels (:obj:`set` of :obj:`str`, optional): The set of dialect labels
            used in the training data in the main model.
            If None, the default labels are used.
            Defaults to None.

        training_model_path (:obj:`str`, optional): Path of training model to be used for inference,
        If none, use defult model for this libaray

        label2index (:obj:`str`, optional): Path of label 2 indexx file to be used for scoring,
        If none, use defult model for this libaray

        max_seq_length (:obj:`int`, optional): maximum sequence length for the model
            
            If None, the default max_seq_length are used.
            Defaults to 256 .

    """
  def __init__(self, labels=None,training_model_path=None,label2index=None,max_seq_length=256,
                batch_size=16 ):
        if labels is None:
            self.labels = _DEFAULT_LABELS
        self._labels_sorted = sorted(labels)
        self.tokenizer = BertTokenizer.from_pretrained(MODEL_PATH_BEGIN_FINETUNE)
        self.batch_size=batch_size
        self.max_seq_length=max_seq_length
        if label2index is None:
          self.label2index= json.load(open(LABEL_2_INDEX_PATH))
        else:
          self.label2index=  json.load(open(label2index))

        if training_model_path is None:
          self.model=BertForSequenceClassification.from_pretrained(MODEL_PATH_, num_labels=len(self.label2index))
          
        else:
          self.model=BertForSequenceClassification.from_pretrained(training_model_path, num_labels=len(self.label2index))

        if torch.cuda.is_available():
          if torch.cuda.device_count() == 1:
            print("Run", "with one GPU")
            self.model = self.model.to(device)
          else:
            n_gpu = torch.cuda.device_count()
            print("Run", "with", n_gpu, "GPUs with max 4 GPUs")
            device_ids = GPUtil.getAvailable(limit = 4)
            torch.backends.cudnn.benchmark = True
            self.model = self.model.to(device)
            self.model = nn.DataParallel(self.model, device_ids=device_ids)
        else:
          print("Run", "with CPU")
          self.model = self.model
        
  def data_prepare_BERT(self,X_train,y_train):
      # Create sentence and label lists
      sentences = X_train.values
      sentences = ['[CLS] ' + sentence + ' [SEP]' for sentence in
                 sentences]
      # Create sentence and label lists
      labels = y_train.values
      labels = [self.labels_json[i] for i in labels]
      # Import the BERT tokenizer, used to convert our text into tokens that correspond to BERT's vocabulary.
      tokenized_texts = [self.tokenizer.tokenize(sent) for sent in sentences]
      # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
      input_ids = [self.tokenizer.convert_tokens_to_ids(x) for x in
                 tokenized_texts]
      # Pad our input seqeunce to the fixed length (i.e., max_len) with index of [PAD] token
      # ~ input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
      pad_ind = self.tokenizer.convert_tokens_to_ids(['[PAD]'])[0]
      input_ids = pad_sequences(
          input_ids,
          maxlen=self.max_seq_length + 2,
          dtype='long',
          truncating='post',
          padding='post',
          value=pad_ind,
          )
      # Create attention masks
      attention_masks = []
      # Create a mask of 1s for each token followed by 0s for padding
      for seq in input_ids:
        seq_mask = [float(i > 0) for i in seq]
        attention_masks.append(seq_mask)
      # Convert all of our data into torch tensors, the required datatype for our model
      inputs = torch.tensor(input_ids)
      labels = torch.tensor(labels)
      masks = torch.tensor(attention_masks)
      return inputs, labels, masks
  def data_prepare_BERT_test(self,X_train):
      # Create sentence and label lists
      sentences = X_train.values
      sentences = ['[CLS] ' + sentence + ' [SEP]' for sentence in
                 sentences]
     
      # Import the BERT tokenizer, used to convert our text into tokens that correspond to BERT's vocabulary.
      tokenized_texts = [self.tokenizer.tokenize(sent) for sent in sentences]
      # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
      input_ids = [self.tokenizer.convert_tokens_to_ids(x) for x in
                 tokenized_texts]
      # Pad our input seqeunce to the fixed length (i.e., max_len) with index of [PAD] token
      # ~ input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
      pad_ind = self.tokenizer.convert_tokens_to_ids(['[PAD]'])[0]
      input_ids = pad_sequences(
          input_ids,
          maxlen=self.max_seq_length + 2,
          dtype='long',
          truncating='post',
          padding='post',
          value=pad_ind,
          )
      # Create attention masks
      attention_masks = []
      # Create a mask of 1s for each token followed by 0s for padding
      for seq in input_ids:
        seq_mask = [float(i > 0) for i in seq]
        attention_masks.append(seq_mask)
      # Convert all of our data into torch tensors, the required datatype for our model
      inputs = torch.tensor(input_ids)
      masks = torch.tensor(attention_masks)
      return inputs,  masks
  def eval(self,X_eval,y_eval, data_set='DEV'):

    """Evaluate the trained model on a given data set.
        Args:
            X_eval (:obj:`np array or pandas series`, optional): loaded data for evaluation.

            y_eval (:obj:`np array or pandas series`, optional): loaded labels for evaluation.

            data_set (:obj:`str`, optional): Name of the provided data set to
                use. This is ignored if data_path is not None. Can be either
                'VALIDATION' or 'TEST'. Defaults to 'VALIDATION'.
        Returns:
            :obj:`dict`: A dictionary mapping an evaluation metric to its
            computed value. The metrics used are accuracy, f1_micro, f1_macro,
            recall_micro, recall_macro, precision_micro and precision_macro.
        """
    validation_inputs, validation_labels, validation_masks = self.data_prepare_BERT(X_eval,y_eval)
    validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
    validation_dataloader = DataLoader(validation_data, batch_size=self.batch_size)
    self.model.eval()
    all_pred=[]
    all_label = []
    with torch.no_grad():
      for i, batch in enumerate(validation_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        input_ids, input_mask, labels = batch
        outputs = self.model(input_ids, input_mask, labels=labels)
        loss, logits = outputs[:2]
        
        probabilities, predicted = torch.max(logits.cpu().data, 1)
        # put all the true labels and predictions to two lists
        all_pred.extend(predicted)
        all_label.extend(labels.cpu())
    accuracy = accuracy_score(all_label, all_pred)
    macro_f1_pos_neg = f1_score(all_label, all_pred,average='macro',labels=[0,1])
    f1score = f1_score(all_label, all_pred, average='macro') 
    recall = recall_score(all_label, all_pred, average='macro')
    precision = precision_score(all_label, all_pred, average='macro')
    # Get scores
    scores = {
        'Sentiment': {
            'accuracy': accuracy,
            'f1_macro': f1score,
            'recall_macro': recall,
            'precision_macro':precision
        }
    }
    return scores

  def predict(self,sentences):
    """Predict the sentiment  probability scores for a given list of
        sentences.
        Args:
            sentences (:obj:`list` of :obj:`str`): The list of sentences.
            output (:obj:`str`): The output label type. Possible values are
                'postive', 'neagtive', 'neutral'.
        Returns:
            :obj:`list` of :obj:`DIDPred`: A list of prediction results,
            each corresponding to its respective sentence.
        """
    if isinstance(sentences, str):
      sentences=pd.Series(sentences)
    validation_inputs, validation_masks = self.data_prepare_BERT_test(sentences)
    validation_data = TensorDataset(validation_inputs, validation_masks)
    validation_dataloader = DataLoader(validation_data, batch_size=self.batch_size)
    self.model.eval()
    all_pred=[]
    result = collections.deque()
    convert = lambda x: x
    
    with torch.no_grad():
      for i, batch in enumerate(validation_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        input_ids, input_mask = batch
        outputs = self.model(input_ids.to(device), input_mask.to(device))
        logits = outputs[:2][0]
        
        
        
        probabilities, predicted = torch.max(outputs[0].cpu().data, 1)
        for i, val in enumerate(self.label2index):
          if i==predicted.item():
            all_pred.extend((val,probabilities.item()))
            result.append(convert(DIDPred(val, logits.cpu().data.numpy().tolist())))
            break
        # put all the true labels and predictions to two lists

        
      
    return list(result)
        


# Testing Production code

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Omdena_sentiment/Dataset/final_text.csv')

In [ ]:
msk = np.random.rand(len(df)) < 0.7
train = df[msk]
test = df[~msk]

In [ ]:
msk = np.random.rand(len(train)) < 0.8
train_new = train[msk]
valid = train[~msk]


In [ ]:
labels_numeric=[0,1,2]

In [ ]:
train_new.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
test.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
valid.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
MARBERT_Sentiment_Classifier=SentimentIdentificationMARBERT(labels_numeric,num_epoch=1)

In [ ]:
MARBERT_Sentiment_Classifier.fine_tune(train_new['final'],train_new['label'],valid['final'],valid['label'])

Some weights of the model checkpoint at /content/MARBERT_pytorch_verison were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the mode

Run with one GPU


In [44]:
MARBERT_Sentiment_Classifier.save_model('/content/drive/MyDrive/Omdena_sentiment/Saved_models/Production/MARBERT')

In [79]:
MARBERT_Sentiment_Classifier.save_label2ind_file('/content/drive/MyDrive/Omdena_sentiment/Saved_models/Production/MARBERT/_labels-dict.json')

AttributeError: ignored

In [46]:
MARBERT_Sentiment_Classifier.eval(test['final'],test['label'])

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Sentiment': {'accuracy': 0.25316363530456815,
  'f1_macro': 0.13469600809306,
  'precision_macro': 0.4177175865249259,
  'recall_macro': 0.3333437744714174}}

In [122]:
Marbert_predictor=SentimentIdentificationMARBERTPrediction(labels_numeric)

Run with one GPU


In [128]:
Marbert_predictor.predict('صفاء الهاشم سيده كويتيه المراه الوحيده حاليا مجلس الامه الكويتي مدافعه شرسه حقوق المراه وحق المواطن الكويتي وتمتلك عقليه اقتصاديه مميزه اختيرت ضمن سيده عربيه مؤsثره مجتمعها')

[DIDPred(top='0', scores=[[4.2006402015686035, -1.7441024780273438, -2.107455015182495]])]